In [2]:
// SETUP
import java.io.File
import java.io.BufferedReader

fun readInputLines(dayIdx: Int): Iterable<String> {
    val fileName = "../data/%02d.txt".format(dayIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

fun readInputLines(dayIdx: Int, testIdx: Int): Iterable<String> {
    val fileName = "../data/%02dt%d.txt".format(dayIdx, testIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

In [113]:
fun solution01() {
    val nums = readInputLines(1).map {it.toInt()}

    fun countIncr(nums: Iterable<Int>) = nums.windowed(2).filter {(a, b) -> a < b}.count()

    val res1 = countIncr(nums)
    val res2 = countIncr(nums.windowed(3).map {it.sum()})

    println("Answer 1: $res1\nAnswer 2: $res2")
}

solution01();

Answer 1: 1557
Answer 2: 1608


In [114]:
enum class Command {
  forward, up, down
}

fun parseLine(line: String): Pair<Command, Int> { 
    val (c, n) = line.split(" ")
    return Pair(Command.valueOf(c), n.toInt())
}

fun step1(pos: Pair<Int, Int>, cmdLine: Pair<Command, Int>): Pair<Int, Int> {
    val (x, depth) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Pair(x, depth - offs)
        Command.down -> Pair(x, depth + offs)
        Command.forward -> Pair(x + offs, depth)
    }
}

fun step2(pos: Triple<Int, Int, Int>, cmdLine: Pair<Command, Int>): Triple<Int, Int, Int> {
    val (x, depth, aim) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Triple(x, depth, aim - offs)
        Command.down -> Triple(x, depth, aim + offs)
        Command.forward -> Triple(x + offs, depth + aim * offs, aim)
    }
}

fun solution02() {
    val commands = readInputLines(2).map {parseLine(it)}
    val (a1, b1) = commands.fold(Pair(0, 0), ::step1)
    val (a2, b2) = commands.fold(Triple(0, 0, 0), ::step2)
    println("Answer 1: ${a1 * b1}\nAnswer 2: ${a2 * b2}")
}

solution02();

Answer 1: 1728414
Answer 2: 1765720035


In [6]:
fun getMostCommon(nums: Iterable<List<Int>>, pos: Int): Int {
    var n0 = 0;
    var n1 = 0;
    for (num in nums) {
        n0 += 1 - num[pos];
        n1 += num[pos];
    }
    if (n1 >= n0) {
        return 1
    } else {
        return 0
    }
}

fun digitsToDecimal(digits: Iterable<Int>): Int {
    return digits.fold(0) {v, d -> v * 2 + d}
}


fun part1(nums: List<List<Int>>): Int {
    var lc = mutableListOf<Int>()
    var mc = mutableListOf<Int>()
    for (i in 0 until nums[0].size) {
        val m = getMostCommon(nums, i)
        mc.add(m)
        lc.add(1 - m)
    }
    return digitsToDecimal(mc) * digitsToDecimal(lc)
}

fun part2(nums: List<List<Int>>): Int {
    var mnums = (0 until nums.size).toList()
    var lnums = (0 until nums.size).toList()
    for (i in 0 until nums[0].size) {
        if (mnums.size > 1) {
            val m = getMostCommon(mnums.map {nums[it]}, i)
            mnums = mnums.filter {nums[it][i] == m }
        }
        if (lnums.size > 1) {
            val m = getMostCommon(lnums.map {nums[it]}, i)
            lnums = lnums.filter {nums[it][i] == 1 - m }
        }
    }
    return digitsToDecimal(nums[mnums[0]]) * digitsToDecimal(nums[lnums[0]])
}

fun solution03() {
    val nums = readInputLines(3).map {it.toCharArray().map {Character.getNumericValue(it)}}
    println("Answer 1: ${part1(nums)}\nAnswer 2: ${part2(nums)}")
}

solution03()

Answer 1: 2972336
Answer 2: 3368358


In [5]:
typealias Board = MutableList<MutableList<Int>>

fun parseGame(lines: Iterable<String>): Pair<List<Int>, List<Board>> {
    val linesIt = lines.iterator()
    val nums = linesIt.next().split(",").map {it.toInt()}
    linesIt.next()
    
    val boards = mutableListOf<Board>()
    var board = mutableListOf<MutableList<Int>>()
    while (linesIt.hasNext()) {
        val line = linesIt.next()
        if (line.isEmpty()) {
            boards.add(board)
            board = mutableListOf<MutableList<Int>>()
        } else {
            board.add(line.trim().split("\\s+".toRegex()).map {it.toInt()}.toMutableList())
        }
    }
    boards.add(board)
    return Pair(nums, boards)
}

fun applyNum(board: Board, num: Int) {
    for (row in board) {
        row.replaceAll {n -> if (n == num) -1 else n}
    }
}

fun isFull(board: Board, pos: Int, is_vert: Boolean): Boolean {
    for (i in 0 until board.size) {
        val n = if (is_vert) board[i][pos] else board[pos][i]
        if (n != -1) {
            return false;
        }
    }
    return true
}

fun hasWinner(board: Board): Boolean {
    return (0 until board.size)
        .any {isFull(board, it, true) || isFull(board, it, false)}
}

fun runSimulation(game: Pair<List<Int>, List<Board>>): List<Int> {
    val (nums, boardsOrig) = game
    val boards = boardsOrig.map {it.toMutableList()}
    var winners = mutableListOf<Int>()
    var scores = mutableListOf<Int>()
    for (n in nums) {
        for ((i, board) in boards.withIndex()) {
            if (winners.contains(i)) {
                continue
            }
            applyNum(board, n)
            if (hasWinner(board)) {
                val s = board.flatten().filter {it != -1}.sum()
                winners.add(i)
                scores.add(s * n)
            }
        }
    } 
    return scores
}

fun solution04() {
    val scores = runSimulation(parseGame(readInputLines(4)))
    println("Answer 1: ${scores[0]}\nAnswer 2: ${scores.last()}")
}

solution04()

Answer 1: 44736
Answer 2: 1827


In [4]:
typealias Point = Pair<Int, Int>
typealias Line = Pair<Point, Point>
    
fun parsePt(pt: String): Point {
    val xy = pt.split(",").map {it.toInt()}
    return Pair(xy[0], xy[1])
}

fun parseLine(line: String): Line {
    val ab = line.split(" -> ").map(::parsePt)
    return Pair(ab[0], ab[1])
}

fun sgn(x: Int): Int {
    if (x == 0) return 0
    if (x < 0) return -1 else return 1
}

fun stroke(a: Point, b: Point, ptmap: HashMap<Point, Int>, skipDiagonals: Boolean) {
    val (x1, y1) = a
    val (x2, y2) = b
    val dx = sgn(x2 - x1)
    val dy = sgn(y2 - y1)
    if (skipDiagonals && dx != 0 && dy != 0) {
        return;
    }
    var cx = x1
    var cy = y1
    while (true) {
        val cpt = Pair(cx, cy)
        ptmap[cpt] = ptmap.getOrDefault(cpt, 0) + 1
        if (cx == x2 && cy == y2) {
            break;
        }
        cx += dx
        cy += dy
    } 
}

fun findNumOverlaps(lines: List<Line>, skipDiagonals: Boolean): Int {
    var ptmap = HashMap<Point, Int>()
    for ((a, b) in lines) {
        stroke(a, b, ptmap, skipDiagonals);
    }
    return ptmap.map {(_, cnt) -> if (cnt >= 2) 1 else 0}.sum()
}

fun solution05() {
    val lines = readInputLines(5).map(::parseLine)
    val res1 = findNumOverlaps(lines, true)
    val res2 = findNumOverlaps(lines, false)
    println("Answer 1: ${res1}\nAnswer 2: ${res2}")
}

solution05()

Answer 1: 7436
Answer 2: 21104


In [22]:
fun getNumSpawned(n: Long, days: Long, spawned: HashMap<Pair<Long, Long>, Long>): Long {
    if (days <= n) {
        return 0
    }
    val key = Pair(n, days)
    if (spawned.containsKey(key)) {
        return spawned.get(key)!!
    }
    var res = (days - n - 1) / 7 + 1
    for (i in 0 .. res) {
        res += getNumSpawned(8, days - n - i * 7 - 1, spawned)
    }
    spawned.put(key, res)
    return res
}

fun sumSpawned(nums: List<Long>, totalDays: Long): Long {
    val spawned = HashMap<Pair<Long, Long>, Long>()
    return nums.map {getNumSpawned(it, totalDays, spawned)}.sum() + nums.size
}

fun solution06() {
    val line = readInputLines(6).first()
    val nums = line.split(",").map {it.toLong()}
    val res1 = sumSpawned(nums, 80)
    val res2 = sumSpawned(nums, 256)
    println("Answer 1: ${res1}\nAnswer 2: ${res2}")
}

solution06();

Answer 1: 386755
Answer 2: 1732731810807


In [23]:
import kotlin.math.*

fun median(nums: List<Double>): Double {
    var nc = nums.sorted()
    val mid = nc.size / 2
    if (nc.size % 2 == 1) {
        return nc[mid]
    } else {
        return (nc[mid - 1] + nc[mid]) / 2.0
    }
}

fun part1(nums: List<Double>): Int {
    val minx = round(median(nums)).toInt()
    return nums.map {abs(it - minx)}.sum().toInt()
}

fun minGradient(p0: Double, f: (Double) -> Double, num_it: Int): Double {
    val EPS = 0.001
    var p = p0
    for (i in 0 until num_it) {
        val pr = f(p + EPS)
        val pl = f(p - EPS)
        val dp = (pr - pl) / (2.0 * EPS)
        p = p - dp * EPS
    }
    return p
}

fun part2(nums: List<Double>): Int {
    val f = {x: Double ->
        var res = 0.0
        for (n in nums) {
            val d = abs(x - n)
            res += d * (d + 1.0) / 2.0
        }
        res
    };
    val minx = minGradient(median(nums), f, 1000)
    return f(round(minx)).toInt()
}

fun solution07() {
    val line = readInputLines(7).first()
    val nums = line.split(",").map {it.toDouble()}
    println("Answer 1: ${part1(nums)}\nAnswer 2: ${part2(nums)}")
}

solution07();

Answer 1: 343468
Answer 2: 96086265


In [56]:
typealias Mapping = MutableList<MutableList<Int>>

val CHARS = "abcdefg"
val DIGITS = arrayOf(
    "abcefg", "cf", "acdeg", "acdfg", "bcdf",
    "abdfg", "abdefg", "acf", "abcdefg", "abcdfg")
val NUM_SEGMENTS = 7

fun normalizeMapping(mapping: Mapping) {
    var changed = true
    while (changed) {
        changed = false
        for (i in 0 until mapping.size) {
            val c = mapping[i]
            if (c.sum() == 1) {
                val p = c.indexOf(1)
                for (j in 0 until NUM_SEGMENTS) {
                    if (j != i && mapping[j][p] == 1) {
                        mapping[j][p] = 0
                        changed = true
                    }
                }
            }
        }
    }
}

fun isValidMapping(mapping: Mapping): Boolean {
    for (i in 0 until NUM_SEGMENTS) {
        if (mapping.map {it[i]}.sum() != 1) {
            return false
        }
    }
    return mapping.map {it.sum()}.all {it == 1}
}

fun getMapping(mapping: Mapping, combs: List<String>, pos: Int): Mapping? {
    println("mapping: ${mapping} pos: ${pos}")
    if (pos == combs.size) {
        normalizeMapping(mapping)
        if (isValidMapping(mapping)) {
            return mapping.toMutableList()
        } else {
            return null
        }
    }

    val comb = combs[pos]
    for (digit in DIGITS) {
        if (digit.length != comb.length) {
            continue
        }
        println("${digit}")
        val mapping1 = mapping.toMutableList()
        for (d in CHARS) {
            if (!digit.contains(d)) {
                for (c in comb) {
                    mapping1[c.code - 'a'.code][d.code - 'a'.code] = 0
                }
            }
        }
        val resMapping = getMapping(mapping1, combs, pos + 1)
        if (resMapping != null) {
            return resMapping
        }
    }
    return null
}

fun decodeDigit(mapping: Mapping, combs: List<String>): Int {
    var res = 0
    for (comb in combs) {
        var scomb = comb.toMutableList()
        for (i in 0 until comb.length) {
            scomb[i] = (mapping[scomb[i].code - 'a'.code].indexOf(1) + 'a'.code).toChar()
        }
        scomb.sort();
        //let s = str::from_utf8(&scomb).unwrap();
        //let digit = DIGITS.iter().position(|&e| e == s).unwrap();
        //res = res * 10 + digit
    }
    return res
}

fun decodeSum(inp: List<List<List<String>>>): Int {
    var res = 0
    for ((combs, toDecode) in inp) {
        var mapping: Mapping = MutableList(NUM_SEGMENTS) { MutableList(NUM_SEGMENTS) {1} }
        val combsSorted = combs.sortedBy { it.length }
        val resMapping = getMapping(mapping, combsSorted, 0)
        res += decodeDigit(resMapping!!, toDecode)
    }
    return res
}

fun solution08() {
    val inp = readInputLines(8, 1).map {it.split("|").map {it.trim().split(" ")}}
    val res1 = inp
        .map {line -> line[1].filter {listOf(2, 3, 4, 7).contains(it.length)}.count()}
        .sum()
    val res2 = decodeSum(inp)
    println("Answer 1: ${res1}\nAnswer 2: ${res2}")
}

solution08()

mapping: [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]] pos: 0
cf
mapping: [[0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]] pos: 1
acf
mapping: [[0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]] pos: 2
bcdf
mapping: [[0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1]] pos: 3
acdeg
mapping: [[0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1]] pos: 4
acdeg
mapping: [[0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1], [1, 0, 


java.lang.NullPointerException
org.jetbrains.kotlinx.jupyter.ReplEvalRuntimeException: 
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:106)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:64)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withHost(repl.kt:598)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:63)
	at org.jetbrains.kotlinx.jupyter.repl.CellExecutor$DefaultImpls.execute$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:421)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl$evalEx$1.invoke(repl.kt:410)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.withEvalContext(repl.kt:374)
	at org.jetbrains.kotlinx.jupyter.ReplForJupyterImpl.evalEx(repl.kt:410)
